### geocode


In [1]:
import numpy as np
import rasterio as rio
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt
from osgeo import gdal, gdal_array
from mintpy.utils import readfile, writefile

In [2]:
#method 1
!/mnt/Backups/gbrench/sw/isce2/src/MintPy/src/mintpy/cli/geocode.py ../data/swatch_27518/Igrams/*/filt*.unw -l ../data/swatch_27518/inputs/geometryRadar.h5 
!/mnt/Backups/gbrench/sw/isce2/src/MintPy/src/mintpy/cli/geocode.py ../data/swatch_27518/Igrams/*/filt*.cor -l ../data/swatch_27518/inputs/geometryRadar.h5 
!/mnt/Backups/gbrench/sw/isce2/src/MintPy/src/mintpy/cli/geocode.py ../data/swatch_27518/inputs/geometryRadar.h5 -l ../data/swatch_27518/inputs/geometryRadar.h5 

number of processor to be used: 1
resampling software: pyresample
read latitude / longitude from lookup table file: ../data/swatch_27518/inputs/geometryRadar.h5
calculate output pixel size using approach 1 (same pixel area before/after resampling)
output pixel size in (lat, lon) in degree: (-4.374737602135722e-05, 6.521692284088274e-05)
output area extent in (S, N, W, E) in degree: (39.119284050389496, 39.3392896044009, -106.83027858014111, -106.28428250211724)
output file row / column number: (5029, 8372)
[1/1] preparing geometry for dest_box: (0, 0, 8372, 5029) --> full src_box: (0, 0, 3142, 9564)
--------------------------------------------------
resampling file: ../data/swatch_27518/Igrams/20230725_20230925/filt_20230725_20230925_snaphu.unw
--------------------------------------------------1/1
reading magnitude in block (0, 0, 3142, 9564) from filt_20230725_20230925_snaphu.unw ...
nearest resampling with pyresample.kd_tree using 1 CPU cores in 43 segments ...
----------------------

In [3]:
# generate xmls
meta={'BANDS' : '1',
      'DATA_TYPE': 'float32',
      'FILE_TYPE': 'hgt',
      'FILE_LENGTH': str(5029), 
      'LENGTH': str(5029),
      'WIDTH': str(8372),
      'length': str(5029),
      'width': str(8372)}

writefile.write_isce_xml(meta, 'geo_filt_20230725_20230925.cor')

meta['BANDS'] = '2'
meta['INTERLEAVE'] = 'BIL'

writefile.write_isce_xml(meta, 'geo_filt_20230725_20230925_snaphu.unw')

write file: geo_filt_20230725_20230925.cor.xml
write file: geo_filt_20230725_20230925.cor.vrt
write file: geo_filt_20230725_20230925_snaphu.unw.xml
write file: geo_filt_20230725_20230925_snaphu.unw.vrt


In [4]:
# function to rewrite coordinates from metadata
def coord_range(ds):
    latrange = np.linspace(float(ds.attrs['Y_FIRST']),
                           ((float(ds.attrs['Y_STEP'])*float(ds.attrs['LENGTH']))+float(ds.attrs['Y_FIRST'])),
                           int(ds.attrs['LENGTH']))
    lonrange = np.linspace(float(ds.attrs['X_FIRST']),
                           ((float(ds.attrs['X_STEP'])*float(ds.attrs['WIDTH']))+float(ds.attrs['X_FIRST'])),
                           int(ds.attrs['WIDTH']))
    return latrange, lonrange

def mintpy2d_to_xarray(fn, crs):
    ds = xr.open_dataset(fn, engine='h5netcdf', phony_dims='sort')
    ds = ds.rename_dims({'phony_dim_0':'y',
                         'phony_dim_1':'x'
                        })
    latrange, lonrange = coord_range(ds)
    ds = ds.assign_coords({'y': ('y', latrange),
                          'x': ('x', lonrange)})
    ds = ds.rio.write_crs(crs)
    
    return ds

In [5]:
# open geocoded data
igram_ds = rioxarray.open_rasterio('geo_filt_20230725_20230925_snaphu.unw.vrt', band_as_variable=True)
igram_ds = igram_ds.rio.write_crs('4326')

cor_ds = rioxarray.open_rasterio('geo_filt_20230725_20230925.cor.vrt', band_as_variable=True)
cor_ds = cor_ds.rio.write_crs('4326')

ds = mintpy2d_to_xarray('geo_geometryRadar.h5', crs='4326')

/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/rioxarray/_io.py:1136: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/rioxarray/_io.py:1136: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/rioxarray/_io.py:1136: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/rioxarray/_io.py:1136: NotGeoreferencedWarning: Dataset

In [6]:
# add to dataset
ds['cor'] = (('y', 'x'), cor_ds.band_1.values)
ds['phase_rad'] = (('y', 'x'), igram_ds.band_2.values)

# convert to meters 
radarWavelength=0.238403545
ds['phase_m'] = ds['phase_rad']*radarWavelength/12.5663706

In [7]:
# export datasets
export_dir = '../data/swatch_27518/geo'
ds.phase_m.rio.to_raster(f'{export_dir}/swatch_27518_20230725_20230925_unw_phase_m.tif')
ds.cor.rio.to_raster(f'{export_dir}/swatch_27518_20230725_20230925_cor.tif')
!mv geo_geometry*.h5 $export_dir

# remove isce datasets
!rm geo_filt*

In [ ]:
f, ax = plt.subplots(figsize=(15, 15))
ax.imshow(ds['phase_m'].values[4000:5000, 500:2500], cmap='RdBu_r', vmin=-0.5, vmax=0.5, interpolation='none')
ax.set_title('InSAR unwrapped phase')
f.tight_layout()